In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

from IPython.display import clear_output
from datetime import datetime

# = = = = = = = = = = = = = = =

import pandas as pd

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

from bs4 import BeautifulSoup
from lxml import etree

# = = = = = = = = = = = = = = =

import json

# = = = = = = = = = = = = = = =

output_correct, output_error = pd.DataFrame(), pd.DataFrame()

# = = = = = = = = = = = = = = =

input_ = pd.read_excel(f'./file/6.url.xlsx',
                       header=0,
                       dtype=str).fillna('')

input_['Year'] = input_['Year'].astype(int)
input_['SKU Count'] = input_['SKU Count'].astype(int)
input_['Page'] = input_['Page'].astype(int)
input_['No'] = input_['No'].astype(int)

input_ = input_.drop_duplicates(['Url'],
                                keep='first',
                                ignore_index=True)
input_ = input_.sort_values(by=['Url'],
                            ascending=[True],
                            ignore_index=True)

input_['No'] = [i+1 for i in range(len(input_))]

print(f'总数量：{len(input_)}')
print()

# = = = = = = = = = = = = = = =


work = Queue()
for a in range(len(input_)):
    work.put_nowait(a)

# = = = = = = = = = = = = = = =

def crawler():
    global output_correct, output_error

    while not work.empty():
        a = work.get_nowait()

        # = = = = = = = = = = = = = = =

        crawler_status = 'error'

        # = = = = = = = = = = = = = = =

        try:
            b = 0
            while True:
                b += 1
                
                try:
                    resp = requests.get(input_.loc[a, 'Url'],
                                        headers=get_header(),
                                        proxies=get_proxy(),
                                        timeout=(10, 10))

                    if resp.status_code == 200:
                        soup = BeautifulSoup(resp.text, 'lxml')
                        html = etree.HTML(str(soup))

                        if html.xpath('//h3/text()')[0].strip() == 'Can\'t find the item you are looking for on this list? Download this':
                            break
            
                        title = html.xpath('//h1[@itemprop="name"]/text()')[0].strip()
                        if html.xpath('//li[@class="global-views-breadcrumb-item"]/a[@data-touchpoint="home"]/text()')[0].strip() == 'Home' and html.xpath('//li[@class="global-views-breadcrumb-item-active"]/text()')[0].strip() == title:
                            break
                except KeyboardInterrupt:
                    break
                except:
                    continue

            # = = = = = = = = = = = = = = =

            if html.xpath('//h3/text()')[0].strip() == 'Can\'t find the item you are looking for on this list? Download this':
                raise

            # = = = = = = = = = = = = = = =

            list_src = html.xpath('//ul[@class="bxslider"]/li[@class="product-details-image-gallery-container"]/noscript/img[@itemprop="image"]/@src') + html.xpath('//div[@class="product-details-image-gallery"]/div[@class="product-details-image-gallery-detailed-image"]/noscript/img[@class="center-block"]/@src')
            list_src = sorted(list(set(list_src)), key=list_src.index)

            # = = = = = = = = = = = = = = =

            dict_src = {}
            for i, src in zip(range(len(list_src)), list_src):
                dict_src[str(i)] = src.strip()

            # = = = = = = = = = = = = = = =

            sku = html.xpath('//span[@itemprop="sku"]/text()')[0].strip()

            # = = = = = = = = = = = = = = =

            dict_description, list_tr = {}, html.xpath('//form[@id="product-details-full-form"]/table[@class="specs-table table table-striped"]/tbody/tr')
            for i, tr in zip(range(len(list_tr)), list_tr):
                dict_description[str(i)] = {tr.xpath('./td[1]/text()')[0].strip(): tr.xpath('./td[2]/text()')[0].strip()}

            # = = = = = = = = = = = = = = =

            list_oem = [text.strip() for text in html.xpath('//form[@id="product-details-full-form"]/div[@data-view="Additional.Detail.Fields"]/div/table[@class="specs-table table table-striped"]/tbody/tr/td[1]/text()')]
            list_oem = sorted(list(set(list_oem)), key=list_oem.index)
            
            oem = ';'.join(list_oem)

            # = = = = = = = = = = = = = = =

            df_temp = pd.DataFrame([{'No': input_.loc[a, 'No'],
                                     'SKU': sku,
                                     'Title': title,
                                     'OEM': oem,
                                     'Picture': '',
                                     'Url': input_.loc[a, 'Url'],
                                     'Json_Src': json.dumps(dict_src),
                                     'Json_Description': json.dumps(dict_description)}])
            
            output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

            # = = = = = = = = = = = = = = =

            crawler_status = 'ok'
        except:
            output_error = pd.concat([output_error, pd.DataFrame([input_.iloc[a]]).reset_index(drop=True)], ignore_index=True).fillna('')

        # = = = = = = = = = = = = = = =

        if work.qsize() % 1_000 == 0 and work.qsize() != 0:
            clear_output()

        print(f'''[状态：{crawler_status}，尝试次数：{b}] - {input_.loc[a, 'No']}.{input_.loc[a, 'Url']}\n[剩余数量：{work.qsize()}] - [当前时间：{datetime.now().strftime('%H:%M:%S')}]\n''')

# = = = = = = = = = = = = = = =

list_task = []
for _ in range(30):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print('输出ing...')
print()
if not output_correct.empty:
    output_correct = output_correct.sort_values(by=['No'],
                                                ascending=[True],
                                                ignore_index=True)
    output_correct.to_excel(f'''./file/7.part-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx''', index=False)
if not output_error.empty:
    output_error = output_error.sort_values(by=['No'],
                                            ascending=[True],
                                            ignore_index=True)
    output_error.to_excel(f'''./file/part_error.xlsx''', index=False)
    print('爬虫存在error')
    print()
print('Done ~')

[状态：ok，尝试次数：3] - 2451.https://www.auveco.com/mitsubishi-drip-moulding-garnish-moulding-clip-19314
[剩余数量：1000] - [当前时间：13:29:44]

[状态：ok，尝试次数：17] - 2442.https://www.auveco.com/miniature-bulb-93-16926
[剩余数量：999] - [当前时间：13:29:44]

[状态：ok，尝试次数：4] - 2448.https://www.auveco.com/mitsu-weatherstrip-retainer-5mmx13mm-hd-size-19325
[剩余数量：998] - [当前时间：13:29:44]

[状态：ok，尝试次数：47] - 2400.https://www.auveco.com/miniature-bulb-161-20285
[剩余数量：997] - [当前时间：13:29:45]

[状态：ok，尝试次数：4] - 2391.https://www.auveco.com/mini-fuse-30-amps-16360
[剩余数量：996] - [当前时间：13:29:45]

[状态：ok，尝试次数：3] - 2453.https://www.auveco.com/mitsubishi-front-bumper-retainer-26mm-stm-lgth-18951
[剩余数量：995] - [当前时间：13:29:45]

[状态：ok，尝试次数：8] - 2457.https://www.auveco.com/mitsubishi-hood-insulation-retainer-22589
[剩余数量：994] - [当前时间：13:29:45]

[状态：ok，尝试次数：38] - 2439.https://www.auveco.com/miniature-bulb-916na-18007
[剩余数量：993] - [当前时间：13:29:46]

[状态：ok，尝试次数：9] - 2456.https://www.auveco.com/mitsubishi-hood-insulation-retainer-22082
[剩余数量：992]